## $\text{Introduction}$

---



$\text{Name : Kartik Srinivas}$


$\text{Roll : ES20BTECH11015}$

$\text{Welcome to the analysis of the Transmitter Problem !}$

1.   
$\text{All the equations have been coded in LaTeX by me}$ 
2.  
$\text{The necessary explanations have been provided }$ 


## $\text{Notation}$

---



$\text{The Notations used are as follows}$



1.   $ \text{The Signal Power Matrix } S \ 𝝐 \ \mathbb{R^{n \times 1}} $
2.   $ \text{The Power Matrix } p \ 𝝐 \ \mathbb{R^{n \times 1}} $

2.    $ \text{The set of Maximum Allowable Transmitter Powers } P_{max} \ 𝝐 \ \mathbb{R^{n \times 1}}$

1.   $ \text{The maximum receiver Powers } P_{rec} \ 𝝐 \ \mathbb{R^{n \times 1}} $
2.   $ \text{The Path gain matrix } G \ 𝝐 \ \mathbb{R^{n \times n}} $

1.   $ \text{The Interference matrix } I \ 𝝐 \ \mathbb{R^{n \times n}} $
 
1.   $ \text{The Groups } Grp \ 𝝐 \ \mathbb{R^{k \times n}} $
2.   $ \text{The Group Maximum } Grp_{max} \ 𝝐 \ \mathbb{R^{k \times 1}} $

1.   $\text{Ratio Parameter } α \ 𝛜 \ \mathbb{R} $








In [28]:
import cvxpy as cp
import numpy as np

G = np.matrix([[1.0, 0.1, 0.2, 0.1, 0.0],
               [0.1,1.0,0.1,0.1,0.0],
               [0.2,0.1,2.0,0.2,0.2],
               [0.1,0.1,0.2,1.0,0.1],
               [0.0,0.0,0.2,0.1,1.0]])

n, m = np.shape(G)

P_max = np.matrix([3.0]*n).T

P_received = np.matrix([5.0,5.0,5.0,5.0,5.0]).T

sigma = np.matrix([0.5,0.5,0.5,0.5,0.5]).T

Group = np.matrix([[1.,1.,0,0,0],[0,0,1.,1.,1.]])

Group_max = np.matrix([4.0,6.0]).T


### $\text{Calculating Signal Powers}$

---
Note that $S = ∑ G_{ii} p_i  = (I_n * G) · p \  $ ($I_n$ is the identity matrix and $*$ is element wise multiplication!) 
Let

$I_n * G = G_{diag} \implies S = G_{diag} · p$

Also note that the interference is just the total received power - the Signal power Received, this takes the form

$I  = (G · p  - S ·p)$

Note that 
\begin{align}
\frac{S_i}{I_i + σ_i} = \frac{G_{diag}(i)^Tp }{(G(i) - S(i))^T p + σ} = \frac{a^Tp + 0 }{c^T p + d}
\end{align}




### $\text{Framing the data}$

---

Aside from the data given, we frame a matrix $Grp$ such that it contains 1's in the respective positions of the groups and 0's everywhere else, for our problem the Grp matrix looks like this (one row per group and 1's in the positions of transmitters):

\\

\begin{align}
           Grp = 
              \begin{bmatrix}
                1 & 1 & 0 & 0 & 0\\
                0 & 0 & 1 & 1 & 1\\
          \end{bmatrix}        
\end{align}

\begin{align}
           Grp \times p = 
              \begin{bmatrix}
                1 & 1 & 0 & 0 & 0\\
                0 & 0 & 1 & 1 & 1\\
          \end{bmatrix} 
          \times 
          \begin{bmatrix} 
           p_1
           \\
           p_2
           \\
           p_3\\
           p_4
           \\
           p_5
          \end{bmatrix} \le Grp_{max}       
\end{align}


### $\text{The Basic Constraints}$

---

The constraints are as follows

\begin{align}
  p \ge 0 
  \\
  p \le P_{max}
  \\
  Grp \times p < Grp_{max}
  \\
  G \times p < P_{rec}
\end{align}





### $\text{The Main Constraint}$

---

Here is the rub, solving the problem directly is difficult. This is because the objective function $O$, is not Convex in nature(It is Quasiconvex) and therefore we must solve this program Manually. We need CVXPY to check whether our **chosen value of $α$ yields a feasible value of p**

\begin{align}
\text{Non Convex O: } max\ (min_i \ ( \frac{S_i}{I_i + σ_i}))
\end{align}

$\text{The Quasi-Convex Formulation:}$


\begin{align}
\text{Quasi Convex O: } min\ (max_i \ ( \frac{I_i + σ_i}{S_i})) = min \ (α)
\\
\text{Additional Constraint: } α \le \frac{I_i + σ_i}{S_i}
\end{align}

We keep investigating correct values of alpha that make this problem feasible



### $\text{The SINR function}$

---

In this function, we will calculate the **Minimum** appropraite value of the parameter $α$ so that the problem is feasible. To do this we must repeatedly check if the problem status is feasible within a partciular interval until we obtain accuracy of a certain value.(Bisection Method)


Here **if the problem is optimal with $L = x$ then the correct  minimum value of alpha lies below the lower bound! so we must reduce x!**


We first check for $L = 0$  the problem is optimal.
Similarly we must choose $U$ to be large enough so that the problem is feasible.

On testing, $L = 0 ,\  U = 10^5$ work!


In [29]:
def sinr(G, P_max, P_received, sigma, Group, Group_max, epsilon = 0.005):

    n, m = np.shape(G)

    In = np.identity(n)
    G_diag = np.multiply(G,In)# Diagonal Matrix containing only G_ii 
    I = G-G_diag # interference power matrix ( without the dot product p, that will come in constraints)


    p = cp.Variable(shape=(n,1))
    best = np.zeros((n,1))


    u = 1e5 #these values work!
    l = 0


    alpha = cp.Parameter(shape=1)

    #  constraints for the bisection feasibility test
    constraints = [I@p + sigma <= alpha*G_diag@p, p <= P_max, p >= 0, G@p <= P_received, Group@p <= Group_max]

    # define objective function, in our case it's constant as only want to test the solution's feasibility
    obj = cp.Minimize(alpha)

    # now check whether the solution lies between u and l
    alpha.value = [u]
    prob = cp.Problem(obj, constraints)
    prob.solve()

    if prob.status != 'optimal':
        # checking optimality of upper bound
        print('No optimal solution within bounds\n')
        return 'Error: no optimal solution within bounds', np.nan, np.nan, np.nan

    alpha.value = [l]
    prob = cp.Problem(obj, constraints)
    prob.solve()

    if prob.status == 'optimal':
#checking for lower bound failure
        print('No optimal solution within bounds\n')
        return 'Error: no optimal solution within bounds', np.nan, np.nan, np.nan

    # Bisection algortithm starts
    maxnumberofLoop = int(1e7)
    for i in range(1,maxnumberofLoop):
  
        alpha.value = np.atleast_1d((u + l)/2.0)

        # test the size of the interval against epsilon
        if u-l <= epsilon:
            break

        # form and solve problem
        prob = cp.Problem(obj, constraints)
        prob.solve()

        # If the problem is feasible u -> α, if not l -> α, best takes the last feasible value as the optimal one as
        # when the tolerance is reached the new α may be out of bounds
        if prob.status == 'optimal':
            u = alpha.value
            best = p.value
        else:
            l = alpha.value

        # final condition to check that the interval has converged to order epsilon
        if u - l > epsilon and i == (maxnumberofLoop-1):
            print("Solution not converged to order epsilon")

    return l, u, float(alpha.value), best

l, u, alpha, best = sinr(G, P_max, P_received, sigma, Group, Group_max)



### **$\text{Final Answer}$**

---
The evaluation will yield an optimal value if $α$ which lies between $L$ and $U$ provided it happens in less that $10^7$ iterations


\begin{align}
 L \le α_{min} \le U
 \\
 |L - U| \le ε = 0.005
\end{align}

The final answer will be 
\begin{align}
α_{min} = max_i \ ( \frac{I_i + σ_i}{S_i}) 
\\
\frac{1}{α_{min}} = min_i \ ( \frac{S_i}{I_i + σ_i}) = SINR = 1.690 \ (Output) 
\end{align}



In [30]:
print('Minimum SINR=',(1/alpha))
print('Power=',best)
In = np.identity(n)
S = np.multiply(G,In)# signal power matrix
I = G-S
max_inv_SINR = np.max((I@best + sigma)/(S@best))
print("Error = ", max_inv_SINR - alpha)
print("Group powers = ", Group@best <= Group_max)
print("P max", best < P_max)
print("P received ", G@best < P_received)


Minimum SINR= 1.69039959697733
Power= [[2.11786574]
 [1.88140368]
 [1.64634514]
 [2.38213007]
 [1.82031606]]
Error =  0.001313779991187758
Group powers =  [[ True]
 [ True]]
P max [[ True]
 [ True]
 [ True]
 [ True]
 [ True]]
P received  [[ True]
 [ True]
 [ True]
 [ True]
 [ True]]


### $\text{Conclusion}$

---

The program output is tested amongst all possible constraints and the error value for the "Additional constraint " Is quite small. This completes the analysis of the problem.